#Import Python libraries
##rdflib - https://pypi.python.org/pypi/rdflib

In [1]:
import os
import rdflib as rdf
#import csv for reading csv files
import csv

#Create new RDF graph

In [2]:
g = rdf.Graph()

#Add namespaces
## Add a namespace for each one in the object model

In [3]:
nidm = rdf.Namespace("http://nidm.nidash.org/")
prov = rdf.Namespace("http://www.w3.org/ns/prov#")
ncit = rdf.Namespace("http://ncitt.ncit.nih.gov/")
nidash = rdf.Namespace("http://purl.org/nidash/nidm/")
fbirn = rdf.Namespace("http://www.birncommunity.org/collaborators/function-birn/")
list(g.namespaces())

[('xml', rdflib.term.URIRef(u'http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef(u'http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('xsd', rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#')),
 ('rdfs', rdflib.term.URIRef(u'http://www.w3.org/2000/01/rdf-schema#'))]

#Bind namespaces to graph

In [4]:
g.bind('nidm', nidm)
g.bind('prov', prov)
g.bind('ncit', ncit)
g.bind('nidash', nidash)
g.bind('fbirn', fbirn)
list(g.namespaces())

[('xml', rdflib.term.URIRef(u'http://www.w3.org/XML/1998/namespace')),
 ('ncit', rdflib.term.URIRef(u'http://ncitt.ncit.nih.gov/')),
 ('rdfs', rdflib.term.URIRef(u'http://www.w3.org/2000/01/rdf-schema#')),
 ('prov', rdflib.term.URIRef(u'http://www.w3.org/ns/prov#')),
 ('nidash', rdflib.term.URIRef(u'http://purl.org/nidash/nidm/')),
 ('nidm', rdflib.term.URIRef(u'http://nidm.nidash.org/')),
 ('rdf', rdflib.term.URIRef(u'http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('abide',
  rdflib.term.URIRef(u'http://fcon_1000.projects.nitrc.org/indi/abide/')),
 ('xsd', rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#'))]

#Create function to create nodes and links in RDF graph

In [5]:
def nidm_add_nodes(graph,line):
    #Create new node for Activity
    g.add((nidash["activity_"+line["subjectID"]], rdf.RDF.type, prov["Activity"]))
    g.add((nidash["activity_"+line["subjectID"]], prov["label"], rdf.Literal("Assessment data collection")))
    #Add new node for Agent
    g.add((nidash["agent_"+line["subjectID"]], rdf.RDF.type, prov["Agent"]))
    g.add((nidash["agent_"+line["subjectID"]], prov["type"], prov["Person"]))
    g.add((nidash["agent_"+line["subjectID"]], ncit["subjectID"], rdf.Literal(line["subjectID"])))
    g.add((nidash["agent_"+line["subjectID"]], ncit["species"], rdf.Literal("Homo sapien")))
    #Create collection for ABIDE WASI variables
    g.add((rdf.Literal("collection_fbirn:NAART_"+line["subjectID"]), rdf.RDF.type, prov["collection"]))
    g.add((rdf.Literal("collection_fbirn:NAART_"+line["subjectID"]), prov["label"], rdf.Literal("fbirn:WASI")))
    #Create an entity for the WASI variables
    g.add((nidash["entity_"+line["SubjectID"]], rdf.RDF.type, prov["Entity"]))
    g.add((nidash["entity_"+line["SubjectID"]], abide["ABIDE_FIQ"], rdf.Literal(line["ABIDE_FIQ"])))
    g.add((nidash["entity_"+line["SubjectID"]], abide["ABIDE_PIQ"], rdf.Literal(line["ABIDE_PIQ"])))
    g.add((nidash["entity_"+line["SubjectID"]], abide["ABIDE_VIQ"], rdf.Literal(line["ABIDE_VIQ"])))
    #Connect WASI variable entity to WASI collection
    g.add((rdf.Literal("collection_abide:WASI_"+line["SubjectID"]), prov["hadMember"], rdf.Literal("entity_"+line["SubjectID"])))
    #Create collection for ABIDE demographics assessment and entity variable and connection
    g.add((rdf.Literal("collection_abide:Demographics_"+line["SubjectID"]), rdf.RDF.type, prov["collection"]))
    g.add((rdf.Literal("collection_abide:Demographics_"+line["SubjectID"]), prov["label"], rdf.Literal("abide:Demographics")))
    g.add((nidash["entity_"+line["SubjectID"]], rdf.RDF.type, prov["Entity"]))
    g.add((nidash["entity_"+line["SubjectID"]], ncit["Age"], rdf.Literal(line["ABIDE_AGE_AT_SCAN"])))
    g.add((nidash["entity_"+line["SubjectID"]], ncit["Handedness"], rdf.Literal(line["ABIDE_HANDEDNESS_CATEGORY"])))
    g.add((rdf.Literal("collection_abide:Demographics_"+line["SubjectID"]), prov["hadMember"], rdf.Literal("entity_"+line["SubjectID"])))
    #Create connections between activity, agent, and collections
    g.add((nidash["activity_"+line["SubjectID"]], prov["wasAssociatedWith"], rdf.Literal("collection_abide:WASI_"+line["SubjectID"])))
    g.add((nidash["activity_"+line["SubjectID"]], prov["wasAssociatedWith"], rdf.Literal("collection_abide:Demographics_"+line["SubjectID"])))
    g.add((nidash["agent_"+line["SubjectID"]], prov["wasAttributedTo"], rdf.Literal("collection_abide:WASI_"+line["SubjectID"])))
    g.add((nidash["agent_"+line["SubjectID"]], prov["wasAttributedTo"], rdf.Literal("collection_abide:Demographics_"+line["SubjectID"])))

#Read in CSV file and convert to RDF graph

In [6]:
input_file = csv.DictReader(open("abide_data.csv"))
for line in input_file:
    nidm_add_nodes(g,line)  

In [7]:
print g.serialize(format='turtle')

@prefix abide: <http://fcon_1000.projects.nitrc.org/indi/abide/> .
@prefix ncit: <http://ncitt.ncit.nih.gov/> .
@prefix nidash: <http://purl.org/nidash/nidm/> .
@prefix nidm: <http://nidm.nidash.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

nidash:activity_CMU_50643 a prov:Activity ;
    prov:label "Assessment data collection" ;
    prov:wasAssociatedWith "collection_abide:Demographics_CMU_50643",
        "collection_abide:WASI_CMU_50643" .

nidash:activity_CMU_50644 a prov:Activity ;
    prov:label "Assessment data collection" ;
    prov:wasAssociatedWith "collection_abide:Demographics_CMU_50644",
        "collection_abide:WASI_CMU_50644" .

nidash:activity_CMU_50645 a prov:Activity ;
    prov:label "Assessment data collection" ;
    prov:wasAssociatedWith "c

#Save RDF Turtle file to disk

In [8]:
with open("ABIDE_to_NIDM_RDFLib.ttl",'w') as f:
    f.write(g.serialize(format='turtle'))

#Create RDF graph from DOT file

In [9]:
from rdflib.tools import rdf2dot
with open("ABIDE_to_NIDM_RDFLib.dot",'w') as f:
    s = rdf2dot.rdf2dot(g, f)


In [10]:
import pydot
g = pydot.graph_from_dot_file('ABIDE_to_NIDM_RDFLib.dot')
g.write_png('ABIDE_to_NIDM_RDFLib.png')

True